In [1]:
import pandas as pd
import numpy as np


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [20]:
data_summer = pd.read_excel("data/2022-2023_Summer.xlsx", "Summer 2022 Structures")
data_fall = pd.read_excel("data/2022-2023_Fall.xlsx", "Fall 2022 Structures")
data_winter = pd.read_excel("data/2022-2023_Winter.xlsx", "By Structure")
data_spring = pd.read_excel("data/2022-2023_Spring.xlsx", "ByStructurePivotTable")